<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">
 
# Logistic Regression
 
_Authors: Multiple_<br>
_Updated: B Rhodes (DC)_
 
---


<a id="learning-objectives"></a>
### Learning Objectives
- Recall how to perform linear regression in scikit-learn.
- Demonstrate why logistic regression is a better alternative for classification than linear regression.
- Understand the concepts of probability, odds, e, log, and log-odds in relation to machine learning.
- Explain how logistic regression works.
- Interpret logistic regression coefficients.
- Use logistic regression with categorical features.
- Compare logistic regression with other models.
- Utilize different metrics for evaluating classifier models.
- Construct a confusion matrix based on predicted classes.

### Lesson Guide
- [Refresher: Fitting and Visualizing a Linear Regression Using scikit-learn](#refresher-fitting-and-visualizing-a-linear-regression-using-scikit-learn)
- [Refresher: Interpreting Linear Regression Coefficients](#refresher-interpreting-linear-regression-coefficients)
- [Predicting a Categorical Response](#predicting-a-categorical-response)
- [Using Logistic Regression for Classification](#using-logistic-regression-for-classification)
- [Probability, e, Log, and Log Odds](#probability-odds-e-log-and-log-odds)
	- [Understanding e and the Natural Logarithm](#understanding-e-and-the-natural-logarithm)
	- [Log Odds](#the-log-odds)
- [What Is Logistic Regression?](#what-is-logistic-regression)
- [Interpreting Logistic Regression Coefficients](#interpreting-logistic-regression-coefficients)
- [Using Logistic Regression With Categorical Features](#using-logistic-regression-with-categorical-features)
- [Comparing Logistic Regression to Other Models](#comparing-logistic-regression-to-other-models)
- [Advanced Classification Metrics](#advanced-classification-metrics)
	- [Accuracy, True Positive Rate, and False Negative Rate](#accuracy-true-positive-rate-and-false-negative-rate)
	- [The Accuracy Paradox](#the-accuracy-paradox)
- [OPTIONAL: How Many Samples Are Needed?](#samples)
- [Lesson Review](#lesson-review)

# Introduction

In this lesson we learn about Logistic Regression, or what is sometimes referred to as Logistic Classification.

"How can a model be both a Regression and a Classification?" you may ask.  


## Discussion

Have you ever had to sort objects, but everything didn't fit perfectly into groups?

Example:
- Movies/Books
- Socks
- Phone apps


Logistic Regression is a classification method that can be used for binary classification or multi-class problems (*i.e.* more than two target classes).

<a id="refresher-fitting-and-visualizing-a-linear-regression-using-scikit-learn"></a>
## Refresher: Fitting and Visualizing a Linear Regression Using scikit-learn
-----
<p style="color: lightblue;"> 20m</p>

Use Pandas to load in the glass attribute data from the UCI machine learning website. The columns are different measurements of properties of glass that can be used to identify the glass type. For detailed information on the columns in this data set, please see the included ```glass.names``` file the in the data folder in this repo.

In [ ]:
# standard data science imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# model & metric imports
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


%matplotlib inline
sns.set(font_scale=1.5);


In [ ]:
# Glass identification data set (comma separated, with header)
url = 'data/glass.csv'

glass_df = pd.read_csv(url)
glass_df.head()

In [ ]:
# display the column headings of the dataframe
list(glass_df.columns)

**Data Dictionary**

- `Id`: number: 1 to 214
- `RI`: refractive index  
- `Na`: Sodium (unit measurement: weight percent in corresponding oxide, as are attributes 4-10)
- `Mg`: Magnesium
- `Al`: Aluminum
- `Si`: Silicon
- `K` : Potassium
- `Ca`: Calcium
- `Ba`: Barium
- `Fe`: Iron
- `Type` : Type of glass:

In [ ]:
# change dataframe columns to lowercase
glass_df.columns = [x.lower() if len(x) < 3 else 'glass_type' for x in glass_df.columns]
#glass_df.columns = ['na','mg','al','si','k','ca','ba','fe','ri','glass_type']

# Reorder the columns - this is really just a convenience step. I prefer the target(s) to be on the end.
glass_df = glass_df[['na','mg','al','si','k','ca','ba','fe','ri','glass_type']]

glass_df.head(2)

**Let's predict `ri`, with only the feature `al`. How could we do it using machine learning?**

<!--
**Answer:** We could frame it as a regression problem, and use a linear regression model with **`al`** as the only feature and **`ri`** as the response.
-->

**How would we visualize this model?**
<!--
**Answer:** Create a scatter plot with **`al`** on the x-axis and **`ri`** on the y-axis, and draw the line of best fit.
-->

In [ ]:
#scatter with regression line
sns.lmplot(x='al', y='ri', data=glass_df);
plt.title('Refractive Index vs Al, w/ Trendline');

**How can we draw this plot (just the points — don't worry about the regression line) without using Seaborn?**

In [ ]:
# Scatter plot using Pandas - fill this with the proper parameters.
glass_df.plot(kind='scatter', x='al', y='ri');
plt.title('Refractive Index vs Al (.plot())');

In [ ]:
# Now use seaborn to make the same plot, without the regression line.
# Use shift-tab to look at the docstring
sns.lmplot(x='al', y='ri', data=glass_df, fit_reg=False);
plt.title('Refractive Index vs Al (seaborn)');

In [ ]:
# Equivalent scatter plot using Matplotlib - Pass in the proper arguments for each command.
plt.scatter(glass_df['al'], glass_df['ri']);
plt.xlabel('al');
plt.ylabel('ri');

To build a linear regression model to predict `ri` using scikit-learn, we will need to Import `LinearRegression` from `linear_model`.

**Using `LinearRegression`, fit a model predicting `ri` from `al` (and an intercept).**

In [ ]:
# Fit a linear regression model (name the model "linreg").
#from sklearn.linear_model import LinearRegression


feature_cols = ['al']
X = glass_df[feature_cols]
y = glass_df['ri']

#Step 1 in linear regression modeling - Instantiate
linreg = LinearRegression()


#Step 2 in linear regression modeling - Fit
linreg.fit(X,y)

**Using the `LinearRegression` object we have fit, create a variable that are our predictions for `ri` for each row's `al` in the data set.**

In [ ]:
# Make predictions for all values of X and add back to the original DataFrame.
y_pred = linreg.predict(X)

# add a new column of y_pred
glass_df['y_pred'] = y_pred

In [ ]:
glass_df.head()

**Plot the predicted `ri` against each `al` as a line.**

In [ ]:
# Plot those predictions connected by a line (try plt.plot()).
# make the line red
x_data = glass_df['al']
y_data = y_pred
plt.plot(x_data, y_data, color='red')
plt.xlabel('al');
plt.ylabel('ri');

_Note the y axis labels when comparing to seaborns plot_

**Plot this regression line with the scatter points on the same chart.**

In [ ]:
# Put the plots together (use a scatter and line graph).
#add the scatter plot here
plt.scatter(glass_df['al'], glass_df['ri'])
plt.plot(x_data, y_data, color='red')
plt.xlabel('al')
plt.ylabel('ri')

<a id="refresher-interpreting-linear-regression-coefficients"></a>
## Refresher: Interpreting Linear Regression Coefficients
---
<p style="color: lightblue;"> 15</p>



Recall the simple linear regression equation is $y = \beta_0 + \beta_1x$

$\beta_0$ is the intercept and $\beta_1$ is, in this case, our coefficient on the `al` predictor.

**Print out the intercept and coefficient values from our fit `LinearRegression` object.**

In [ ]:
# assign intercept and coefficient to a variable, beta_0, beta_1, respectively.
# Remember that the coefficient(s) are in a numpy array.
beta_0 = linreg.intercept_
beta_1 = linreg.coef_

# print out the intercept and coefficient(s) for the LinearRegression object we instantiated above
print(f'y-Intercept: {beta_0}')
print(f'Coefficient: {beta_1}')




**Manually compute the predicted value of `ri` when `al=2.0` using the regression equation.**

In [ ]:
# Compute prediction for al=2 using the equation.
#  y = beta_0 + beta_1 * x
y = beta_0 + beta_1 * 2
print(f'By hand: Refractive Index for Al = 2: {y}')

**Confirm that this is the same value we would get when using the built-in `.predict()` method of the `LinearRegression` object.**

In [ ]:
# Compute prediction for al=2 using the predict method.
al_2 = np.array([2]).reshape(-1,1)
y_pred = linreg.predict(al_2)
print(f'By .predict(): Refractive Index for Al = 2: {y_pred}')

In [ ]:
# Compute prediction for al=2 through 8 using the predict method.
al_2 = np.array([2,3,4,5,6,7,8]).reshape(-1,1)
y_pred = linreg.predict(al_2)
print(f'By .predict(): Refractive Index for Al = 2 thru 8:\n {y_pred}')

In [ ]:
np.array([2,3,4,5,6,7,8]).shape

In [ ]:
al_2.shape



In [ ]:
# print out the intercept and coefficient(s) for the LinearRegression object we instantiated above
print(f'y-Intercept: {beta_0}')
print(f'Coefficient: {beta_1}')

In [ ]:
X.shape

In [ ]:
y_pred.shape

**Coefficient interpretation:** A 1-unit increase in `al` is associated with a ~0.0025-unit decrease in `ri`.

**Intercept interpretation:** When `al = 0`, the estimated value of `ri` is 1.52194533024.

---

<a id="predicting-a-categorical-response"></a>
## Predicting a Single Categorical Response
<p style="color: lightblue;"> 15min</p>

---

Linear regression is appropriate when we want to predict the value of a continuous target/response variable, but what about when we want to predict membership in a class or category?

**Examine the glass type column in the data set. What are the counts in each category?**

In [ ]:
glass_df.head()

In [ ]:
# Examine glass_type.
glass_df['glass_type'].value_counts().sort_index()

Say these types are subdivisions of broader glass types:

> **Window glass:** types 1, 2, and 3

> **Household glass:** types 5, 6, and 7

**Create a new `household` column that indicates whether or not a row is household glass, coded as 1 or 0, respectively.**

In [ ]:
# Types 1, 2, 3 are window glass.
# Types 5, 6, 7 are household glass.
glass_df['household'] = glass_df['glass_type'].map({1:0, 2:0, 3:0, 5:1, 6:1, 7:1})
glass_df.head()

In [ ]:
glass_df.tail()

Let's change our task, so that we're predicting the `household` category using `al`. Let's visualize the relationship to figure out how to do this.

**Make a scatter plot comparing `al` and `household`.**

In [ ]:
plt.scatter(glass_df['al'], glass_df['household'])
plt.xlabel('al')
plt.ylabel('household')

**Fit a new `LinearRegression` predicting `household` from `al`.**

Let's draw a regression line like we did before:

In [ ]:
# Fit a linear regression model and store the predictions in a new column.
feature_cols = ['al']

# assign features & target
X = glass_df[feature_cols] 
y = glass_df['household']

#fit the model
linreg.fit(X,y)

# assign the predictions to a new column
glass_df['household_pred'] =  linreg.predict(X)

In [ ]:
glass_df.head(2)

In [ ]:
# Scatter plot that includes the regression line
plt.scatter(glass_df['al'], glass_df['household'])
plt.plot(glass_df['al'], glass_df['household_pred'], color='red')
plt.xlabel('al')
plt.ylabel('household');

If **al=3**, what class do we predict for household? **1**

If **al=1.5**, what class do we predict for household? **0**

We predict the 0 class for **lower** values of al, and the 1 class for **higher** values of al. What's our cutoff value? Around **al=2**, because that's where the linear regression line crosses the midpoint between predicting class 0 and class 1.

Therefore, we'll say that if **household_pred >= 0.5**, we predict a class of **1**, else we predict a class of **0**.

**Using this threshold, create a new column of our predictions for whether a row is household glass.**

In [ ]:
# EXAMPLE to understand np.where
nums = np.array([5, 15, 8])

# np.where returns the first value if the condition is True, and the second value if the condition is False.
np.where(nums > 10, 'big', 'small')

In [ ]:
# Transform household_pred to 1 or 0.
glass_df['household_pred_class'] = np.where(glass_df['household_pred'] >= 0.5, 1, 0)
glass_df.head()

In [ ]:
glass_df.tail()

In [ ]:
glass_df.describe().T

**Plot a line that shows our predictions for class membership in household vs. not.**

In [ ]:
# sort so we can have a continuous line
glass_df.sort_values('al', inplace=True)

# Plot the class predictions.
plt.scatter(glass_df['al'], glass_df['household'])

plt.xlabel('al')
plt.ylabel('household')

plt.plot(glass_df['al'], glass_df['household_pred_class'], color='red')

<a id="using-logistic-regression-for-classification"></a>
## Using Logistic Regression for Classification

<p style="color: lightblue;"> 10min</p>

---


For the problem above logistic regression is a more appropriate method than linear regression. The values output from a linear regression cannot be interpreted as probabilities of class membership since their values can be greater than 1 and less than 0. Logistic regression, on the other hand, ensures that the values output as predictions can be interpreted as probabilities of class membership.

**We imported the `LogisticRegression` class from `linear_model` above. Now we'll fit the logistic regression model predicting `household` from `al` using the standard steps.**

In [ ]:
# Fit a logistic regression model and store the class predictions.
# We already imported this above. It's here just for reference

# from sklearn.linear_model import LogisticRegression


feature_cols = ['al']
X = glass_df[feature_cols]
y = glass_df['household']

#step 1: instantiate
logreg = LogisticRegression()


#step 2 fit
logreg.fit(X,y)

#step 3: use features to predict & assign to variable
type_pred = logreg.predict(X)

**Plot the predicted class using the logistic regression as we did for the linear regression predictions above.**

As you can see, the class predictions are the same.

In [ ]:
# Plot the class predictions.
plt.scatter(glass_df['al'], glass_df['household'])
plt.plot(glass_df['al'], type_pred, color='red')
plt.xlabel('al')
plt.ylabel('household');

What if we wanted the predicted probabilities instead of just the class predictions, to understand how confident we are in a given prediction?

**Using the built-in `.predict_proba()` function, examine the predicted probabilities for the first handful of rows of `X`.**

In [ ]:
logreg.predict_proba(X)

In [ ]:
glass_df.head(10)

Sklearn orders the columns according to our class labels. The two-column output of `predict_proba` returns a column for each class of our `household` variable. The first column is the probability of `household=0` for a given row, and the second column is the probability of `household=1`.

**Store the predicted probabilities of class=1 in its own column in the data set.**

In [ ]:
# Store the predicted probabilities of class 1.
glass_df['household_pred_prob'] = logreg.predict_proba(X)[:, 1]

In [ ]:
glass_df.tail(10)

**Plot the predicted probabilities as a line on our plot (probability of `household=1` as `al` changes).**

In [ ]:
# Plot the predicted probabilities.
plt.scatter(glass_df['al'], glass_df['household'])
plt.plot(glass_df['al'], glass_df['household_pred_prob'], color='red')
plt.xlabel('al')
plt.ylabel('household');

In [ ]:
# Examine some example predictions.
#print predictions for 1-3. use a for loop.
print(f'Predicted Probabilities for each class:')
for i in range(1,4):
    print(logreg.predict_proba(np.array(i).reshape(-1,1)))



In [ ]:
# get the accuracy - note .score() generates the default score appropriate for the model.
# In this case since logistic regression is a classification method it uses accuracy as the default metric.
acc = logreg.score(X,y)

# Compute the accuracy using the .score() method.
print(f'Accuracy: {acc}')

### Exercise 1:

- Build and train a logistic regression model.
- Select 2 features for your feature matrix, $X$
- use `train_test_split()` with `random_state=1` and the default split.
- $y$ will remain the same `glass_df['household']`
- Evaluate the model with `.score()`.

In [ ]:
# A: Take 5min and build a logistic regression model using 2 features from the dataframe.
# the target y is the same as before.

# A:

#Assign the features and targets to appropriate variables
X = None
y = None

# split the data into training and testing data


# Instantiate 
logr = None

# fit the data - aka generate the model

# Make predictions with the test data

y_pred = None

# Evaluate the performance using .score() method.

acc = None
print(f'Accuracy: {acc}')

<a id="probability-odds-e-log-and-log-odds"></a>
## Probability, e, Log, and Log Odds
---

To understand how logistic regression predicts the probability of class membership we need to start by understanding the relationship between probability, odds ratios, and log odds ratios. This is because logistic regression predicts log odds and so reading log odds is extremely useful for interpreting logistic regression.

### Probability
Probability is the liklihood of an event occurring. Say we have 10 balls, 7 red, 3 green in a bag. You are asked, without looking, to reach into the bag and select 1 ball. The probablity of picking a red ball is 7/10 (0.7) and the probability of picking a green ball is 3/10 (0.3). Remember that probabilities always fall between 0 and 1, with 1 being certain an event will occur and 0 being certain the event will *not* occur. Probalities always add up to 1. In this case, 

$$P(red) + P(green) = 0.7 + 0.3 = 1.0$$

In general, probability of an outcome is expressed as:

$$probability = \frac {one\ outcome} {all\ outcomes}$$

### Odds
Odds are related to probability and are another way of expressing the liklihood of an event occurring. However, they have a slightly different formulation. They are 

$$odds = \frac {one\ outcome} {all\ other\ outcomes}$$

It is often useful to think of the numeric odds as a ratio. For example, 5/1 = 5 odds is "5 to 1" -- five wins for every one loss (e.g. of six total plays). 2/3 odds means "2 to 3" -- two wins for every three losses (e.g. of five total plays).

Examples:

- Dice roll of 1: probability = 1/6, odds = 1/5
- Even dice roll: probability = 3/6, odds = 3/3 = 1
- Dice roll less than 5: probability = 4/6, odds = 4/2 = 2

### Probability vs Odds
Each of these can be expressed as a function of the other.

$$odds = \frac {probability} {1 - probability}$$

$$probability = \frac {odds} {1 + odds}$$

**As an example we can create a table of probabilities vs. odds, as seen below.**

In [ ]:
# Create a table of probability versus odds.
prob_odds_df = pd.DataFrame({'probability':[0.1, 0.2, 0.25, 0.5, 0.6, 0.8, 0.9]})
prob_odds_df['odds'] = prob_odds_df.probability / (1 - prob_odds_df.probability)
prob_odds_df

<a id="understanding-e-and-the-natural-logarithm"></a>
### Understanding e and the Natural Logarithm

What is e? It is the base rate of growth shared by all continually growing processes:

e is the irrational base of the natural log `ln`.

- 2.718281828459

In [ ]:
# Exponential function: e^1
e = np.exp(1)
e

What is a (natural) log? It gives you the time needed to reach a certain level of growth:

In [ ]:
# Time needed to grow 1 unit to 2.718 units
# ln e = 1
np.log(2.718281828459) # very close to previous value

In [ ]:
np.log(e)

It is also the inverse of the exponential function:

In [ ]:
# e^5
np.exp(5)

In [ ]:
# np.exp(5)
2.7182818**5

In [ ]:
# Taking the log of the exponential returns back to original input
np.log(np.exp(5))

Lets take one of our odds from out table and walk through how it works.

In [ ]:
# Odds = 0.25
# ln 0.25 = -1.38629436
np.log(0.25)

In [ ]:
print(e**-1.3862943611198906)
print(np.exp(-1.3862943611198906))

<a id="the-log-odds"></a>

When we take the logarithm of the odds, we get what is known as the **log odds**. This may seem like an arbitrary transformation, but it has an important property: The log odds has the range $[-\infty, \infty]$. This is not true for the odds ratio, which can never be a negative number.

In [ ]:
# Add log odds to the table.
prob_odds_df['logodds'] = np.log(prob_odds_df['odds'])
prob_odds_df

<a id="what-is-logistic-regression"></a>
## What Is Logistic Regression?
---

Linear regression: Continuous response is modeled as a linear combination of the features.

$$y = \beta_0 + \beta_1x$$

Logistic regression: Log odds of a categorical response being "true" (1) is modeled as a linear combination of the features.

$$\log \left({p\over 1-p}\right) = \beta_0 + \beta_1x$$

This is called the logit function.

Set $z = {\beta_0 + \beta_1x}$ and take the exponential of each side

$$e^{\log \left({p\over 1-p}\right)} = e^z$$

which gives:

$$ {p\over 1-p} = e^z$$

Solving for $p$ gives:

$$ p  = e^z - p*e^z$$

$$ p + p*e^z  = e^z$$

$$ p(1 + e^z)  = e^z$$

$$ p = \frac{e^z}{(1 + e^z)}$$


Subsitute for z gives the equation for probability, $p$ as the logistic function.

$$p = \frac{e^{\beta_0 + \beta_1x}} {1 + e^{\beta_0 + \beta_1x}}$$

In other words:

- Logistic regression outputs the probabilities of a specific class.
- Those probabilities can be converted into class predictions.

The logistic function has some nice properties:

- Takes on an "s" shape
- Output is bounded by 0 and 1

We have covered how this works for binary classification problems (two response classes). But what about multi-class classification problems (more than two response classes)?

- The most common solution for classification models is "one-vs-all" (also known as "one-vs-rest"): Decompose the problem into multiple binary classification problems.
- Multinomial logistic regression, on the other hand, can solve this as a single problem, but how this works is beyond the scope of this lesson.

<a id="interpreting-logistic-regression-coefficients"></a>
## Interpreting Logistic Regression Coefficients

---

Logistic regression coefficients are not as immediately interpretable as the coefficients from a linear regression. To interpret the coefficients we need to remember how the formulation for logistic regression differs from linear regression.

**First let's plot our logistic regression predicted probability line again.**

In [ ]:
# Plot the predicted probabilities again.
plt.scatter(glass_df['al'], glass_df['household'])
plt.plot(glass_df['al'], glass_df['household_pred_prob'], color='red')
plt.xlabel('al')
plt.ylabel('household')

Remember:

$$\log \left({p\over 1-p}\right) = \beta_0 + \beta_1x$$

That means we'll get out the log odds if we compute the intercept plus the coefficient times a value for `al`.

**Compute the log odds of `household` when `al=2`.**

In [ ]:
# Compute predicted log odds for al=2 using the equation.
logodds = (logreg.intercept_ + logreg.coef_[0] * 2)
logodds

Now that we have the log odds, we will need to go through the process of converting these log odds to probability.

**Convert the log odds to odds, then the odds to probability.**

In [ ]:
# Convert log odds to odds.
odds = np.exp(logodds)
print(odds)

In [ ]:
# Convert odds to probability.
prob = odds/(1 + odds)
print(prob)

This finally gives us the predicted probability of `household=1` when `al=2`. You can confirm this is the same as the value you would get out of the `.predict_proba()` method of the sklearn object.

In [ ]:
# Compute predicted probability for al=2 using the predict_proba method.
logreg.predict_proba(np.array([2, 3]).reshape(-1,1))[:, 1]

In [ ]:
# Examine the coefficient for al.
list(zip(feature_cols, logreg.coef_[0]))

In [ ]:
# Print the intercept.
print(logreg.intercept_)

**Interpretation:** A 1-unit increase in `al` is associated with a 2.01-unit increase in the log odds of `household`.

In [ ]:
# Increasing al by 1 (so that al=3)
logodds = -6.00934605 + 3.1151792681570165*4
odds = np.exp(logodds)
prob = odds/(1 + odds)
prob

In [ ]:
# Compute predicted probability for al=3 using the predict_proba method.

logreg.predict_proba(np.array(4).reshape(-1,1))

**Bottom line:** Positive coefficients increase the log odds of the response (and thus increase the probability), and negative coefficients decrease the log odds of the response (and thus decrease the probability).

In [ ]:
# Examine the intercept.
logreg.intercept_

**Intercept interpretation:** For an `al` value of 0, the log-odds of `household` is -4.12790736.

In [ ]:
# Convert log odds to probability.
logodds = logreg.intercept_
odds = np.exp(logodds)
prob = odds/(1 + odds)
print(f'log odds: {logodds}')
print(f'probability: {prob}')


That makes sense from the plot above, because the probability of `household=1` should be very low for such a low `al` value.

![Logistic regression beta values](./assets/logistic_betas.png)

Changing the $\beta_0$ value shifts the curve horizontally, whereas changing the $\beta_1$ value changes the slope of the curve.

<a id="comparing-logistic-regression-to-other-models"></a>
## Comparing Logistic Regression to Other Models
---

Advantages of logistic regression:

- Highly interpretable (if you remember how).
- Model training and prediction are fast.
- No tuning is required (excluding regularization).
- Features don't need scaling.
- Can perform well with a small number of observations.
- Outputs well-calibrated predicted probabilities.

Disadvantages of logistic regression:

- Presumes a linear relationship between the features and the log odds of the response.
- Performance is (generally) not competitive with the best supervised learning methods.
- Can't automatically learn feature interactions.

<a id="advanced-classification-metrics"></a>
## Advanced Classification Metrics

---

When we evaluate the performance of a logistic regression (or any classifier model), the standard metric to use is accuracy: How many class labels did we guess correctly? However, accuracy is only one of several metrics we could use when evaluating a classification model.

$$Accuracy = \frac{total~predicted~correct}{total~predicted}$$

Accuracy alone doesn’t always give us a full picture.

If we know a model is 75% accurate, it doesn’t provide any insight into why the 25% was wrong.

Consider a binary classification problem where we have 165 observations/rows of people who are either smokers or nonsmokers.

<table style="border: none">
<tr style="border: none">
    <td style="border: none; vertical-align: bottom">n = 165</td>
    <td style=""><b>Predicted: No</b></td>
    <td style=""><b>Predicted: Yes</b></td>
</tr>
<tr>
    <td><b>Actual: No</b></td>
    <td style="text-align: center"></td>
    <td style="text-align: center"></td>
    <td style="text-align: center"></td>
</tr>
<tr>
    <td><b>Actual: Yes</b></td>
    <td style="text-align: center"></td>
    <td style="text-align: center"></td>
    <td style="text-align: center"></td>
</tr>
<tr style="border: none">
    <td style="border: none"></td>
    <td style="text-align: center"></td>
    <td style="text-align: center"></td>
</tr>

</table>

There are 60 in class 0, nonsmokers, and 105 observations in class 1, smokers
<table style="border: none">
<tr style="border: none">
    <td style="border: none; vertical-align: bottom">n = 165</td>
    <td style=""><b>Predicted: No</b></td>
    <td style=""><b>Predicted: Yes</b></td>
</tr>
<tr>
    <td><b>Actual: No</b></td>
    <td style="text-align: center"></td>
    <td style="text-align: center"></td>
    <td style="text-align: center">60</td>
</tr>
<tr>
    <td><b>Actual: Yes</b></td>
    <td style="text-align: center"></td>
    <td style="text-align: center"></td>
    <td style="text-align: center">105</td>
</tr>
<tr style="border: none">
    <td style="border: none"></td>
    <td style="text-align: center"></td>
    <td style="text-align: center"></td>
</tr>

</table>

We have 55 predictions of class, predicted as nonsmokers, and 110 of class 1, predicted to be smokers.

<table style="border: none">
<tr style="border: none">
    <td style="border: none; vertical-align: bottom">n = 165</td>
    <td style=""><b>Predicted: No</b></td>
    <td style=""><b>Predicted: Yes</b></td>
</tr>
<tr>
    <td><b>Actual: No</b></td>
    <td style="text-align: center"></td>
    <td style="text-align: center"></td>
    <td style="text-align: center">60</td>
</tr>
<tr>
    <td><b>Actual: Yes</b></td>
    <td style="text-align: center"></td>
    <td style="text-align: center"></td>
    <td style="text-align: center">105</td>
</tr>
<tr style="border: none">
    <td style="border: none"></td>
    <td style="text-align: center">55</td>
    <td style="text-align: center">110</td>
</tr>

</table>

- **True positives (TP):** These are cases in which we predicted yes (smokers), and they actually are smokers.
- **True negatives (TN):** We predicted no, and they are nonsmokers.
- **False positives (FP):** We predicted yes, but they were not actually smokers. (This is also known as a "Type I error.")
- **False negatives (FN):** We predicted no, but they are smokers. (This is also known as a "Type II error.")
<table style="border: none">
<tr style="border: none">
    <td style="border: none; vertical-align: bottom">n = 165</td>
    <td style=""><b>Predicted: No</b></td>
    <td style=""><b>Predicted: Yes</b></td>
</tr>
<tr>
    <td><b>Actual: No</b></td>
    <td style="text-align: center">TN = 50</td>
    <td style="text-align: center">FP = 10</td>
    <td style="text-align: center">60</td>
</tr>
<tr>
    <td><b>Actual: Yes</b></td>
    <td style="text-align: center">FN = 5</td>
    <td style="text-align: center">TP = 100</td>
    <td style="text-align: center">105</td>
</tr>
<tr style="border: none">
    <td style="border: none"></td>
    <td style="text-align: center">55</td>
    <td style="text-align: center">110</td>
</tr>

</table>

**Categorize these as TP, TN, FP, or FN:**

Try not to look at the answers above.
    
- We predict nonsmoker, but the person is a smoker.
- We predict nonsmoker, and the person is a nonsmoker.
- We predict smoker and the person is a smoker.
- We predict smoker and the person is a nonsmoker.

<!--ANSWER
- FN
- TN
- TP
- FP
-->

Take a look at the [confusion matrix method in *sci-kit learn*](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).

<a id="accuracy-true-positive-rate-and-false-negative-rate"></a>
### Accuracy, True Positive Rate, and False Negative Rate

**Accuracy:** Overall, how often is the classifier correct?

<span>
    (<span style="color: green">TP</span>+<span style="color: red">TN</span>)/<span style="color: blue">total</span> = (<span style="color: green">100</span>+<span style="color: red">50</span>)/<span style="color: blue">165</span> = 0.91
</span>

<table style="border: none">
<tr style="border: none">
    <td style="border: none; vertical-align: bottom; color: blue">n = 165</td>
    <td style=""><b>Predicted: No</b></td>
    <td style=""><b>Predicted: Yes</b></td>
</tr>
<tr>
    <td><b>Actual: No</b></td>
    <td style="text-align: center; background-color: red">TN = 50</td>
    <td style="text-align: center">FP = 10</td>
    <td style="text-align: center">60</td>
</tr>
<tr>
    <td><b>Actual: Yes</b></td>
    <td style="text-align: center">FN = 5</td>
    <td style="text-align: center; background-color: green">TP = 100</td>
    <td style="text-align: center">105</td>
</tr>
<tr style="border: none">
    <td style="border: none"></td>
    <td style="text-align: center">55</td>
    <td style="text-align: center">110</td>
</tr>

</table>

**True positive rate (TPR)** asks, “Out of all of the target class labels, how many were accurately predicted to belong to that class?”

For example, given a medical exam that tests for cancer, how often does it correctly identify patients with cancer?

<span>
<span style="color: green">TP</span>/<span style="color: blue">actual yes</span> = <span style="color: green">100</span>/<span style="color: blue">105</span> = 0.95
</span>

<table style="border: none">
<tr style="border: none">
    <td style="border: none; vertical-align: bottom">n = 165</td>
    <td style=""><b>Predicted: No</b></td>
    <td style=""><b>Predicted: Yes</b></td>
</tr>
<tr>
    <td><b>Actual: No</b></td>
    <td style="text-align: center">TN = 50</td>
    <td style="text-align: center">FP = 10</td>
    <td style="text-align: center">60</td>
</tr>
<tr>
    <td><b>Actual: Yes</b></td>
    <td style="text-align: center">FN = 5</td>
    <td style="text-align: center;background-color: green">TP = 100</td>
    <td style="text-align: center;color: blue">105</td>
</tr>
<tr style="border: none">
    <td style="border: none"></td>
    <td style="text-align: center">55</td>
    <td style="text-align: center">110</td>
</tr>

</table>

**False positive rate (FPR)** asks, “Out of all items not belonging to a class label, how many were predicted as belonging to that target class label?”

For example, given a medical exam that tests for cancer, how often does it trigger a “false alarm” by incorrectly saying a patient has cancer?

<span>
<span style="color: orange">FP</span>/<span style="color: blue">actual no</span> = <span style="color: orange">10</span>/<span style="color: blue">60</span> = 0.17
</span>

<table style="border: none">
<tr style="border: none">
    <td style="border: none; vertical-align: bottom">n = 165</td>
    <td style=""><b>Predicted: No</b></td>
    <td style=""><b>Predicted: Yes</b></td>
</tr>
<tr>
    <td><b>Actual: No</b></td>
    <td style="text-align: center">TN = 50</td>
    <td style="text-align: center;background-color: orange">FP = 10</td>
    <td style="text-align: center;color:blue">60</td>
</tr>
<tr>
    <td><b>Actual: Yes</b></td>
    <td style="text-align: center">FN = 5</td>
    <td style="text-align: center">TP = 100</td>
    <td style="text-align: center">105</td>
</tr>
<tr style="border: none">
    <td style="border: none"></td>
    <td style="text-align: center">55</td>
    <td style="text-align: center">110</td>
</tr>

</table>

**Can you see that we might weigh TPR AND FPR differently depending on the situation?**

- Give an example when we care about TPR, but not FPR.
- Give an example when we care about FPR, but not TPR.

<!--
ANSWER:
- During an initial medical diagnosis, we want to be sensitive. We want initial screens to come up with a lot of true positives, even if we get a lot of false positives.
- If we are doing spam detection, we want to be precise. Anything that we remove from an inbox must be spam, which may mean accepting fewer true positives.
-->

**More Trade-Offs**

The true positive and false positive rates gives us a much clearer picture of where predictions begin to fall apart.

This allows us to adjust our models accordingly.

**Below we will load in some data on admissions to college.**

In [ ]:
#import pandas as pd
#from sklearn import linear_model, model_selection, metrics

admissions = pd.read_csv('data/admissions.csv')
admissions = admissions.dropna()
admissions.head(2)

In [ ]:
# Get dummy variables for prestige.
admissions = admissions.join(pd.get_dummies(admissions['prestige'], prefix='prestige'))
admissions.head(2)

**We can predict the `admit` class from `gre` and use a train-test split to evaluate the performance of our model on a held-out test set.**

In [ ]:
X = admissions[['gre']]
y = admissions['admit']

# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=46)

# assign fit results to a variable, apply weak regularization C=1e9
logit_simple = LogisticRegression(C=1e9).fit(X_train, y_train)

In [ ]:
admissions['admit'].value_counts(normalize=True)

**Recall that our "baseline" accuracy is the proportion of the majority class label.**

In [ ]:
1. - y_test.mean()

In [ ]:
# What is our accuracy on the test set?
print(f'Accuracy: {np.mean(y_test == logit_simple.predict(X_test))}')

**Create a confusion matrix of predictions on our test set using `metrics.confusion_matrix`**.

In [ ]:
# Get probability predictions.
logit_pred_proba = logit_simple.predict_proba(X_test)[:,1]


Note that the below `.ravel()` code will only work with a [binary classification](https://en.wikipedia.org/wiki/Binary_classification) problem (two classes, i.e. 'is pregnant' or 'is not pregnant'). For a multiclass problem, one would need either need to modify the variables 'catching' the output of the `metric.confusion_matrix()` function to equal the number of classes in their study, or consider a [OvA](http://mlwiki.org/index.php/One-vs-All_Classification) approach to coerce multiclass into a binary class output.

In [ ]:
# print out the confusion matrix with a 0.5 threshold
print(f'Confusion Matrix:\n {confusion_matrix(y_true=y_test, y_pred=logit_pred_proba > .5)}')

In [ ]:
# .ravel() flattens out a matrix into 1-d array.
tn, fp, fn, tp = confusion_matrix(y_true=y_test, y_pred=logit_pred_proba > .5).ravel() 
(tn, fp, fn, tp)

In [ ]:
# Lower the threshold - how does the confusion matrix change?
print(f'Confusion Matrix:\n {confusion_matrix(y_true=y_test, y_pred=logit_pred_proba > .3)}')

**Answer the following:**

- What is our accuracy on the test set?
- True positive rate?
- False positive rate?

- What changed when we lowered the threshold for predicting the positive class?
    - what elements of the confusion matrix changed.
    - how does that impact the metrics?

<!--
ANSWER: This will depend on the data:
Accuracy: 64%
TPR: 0
FPR: 0
-->

In [ ]:
# Answer here:


A good classifier would have a true positive rate approaching 1 and a false positive rate approaching 0.

In our smoking problem, this model would accurately predict all of the smokers as smokers and not accidentally predict any of the nonsmokers as smokers.

### Trading True Positives and True Negatives

By default, and with respect to the underlying assumptions of logistic regression, we predict a positive class when the probability of the class is greater than .5 and predict a negative class otherwise.

What if we decide to use .3 as a threshold for picking the positive class? Is that even allowed?

This turns out to be a useful strategy. By setting a lower probability threshold we will predict more positive classes. Which means we will predict more true positives, but fewer true negatives.

Making this trade-off is important in applications that have imbalanced penalties for misclassification.

The most popular example is medical diagnostics, where we want as many true positives as feasible. For example, if we are diagnosing cancer we prefer to have false positives, predict a cancer when there is no cancer, that can be later corrected with a more specific test.

We do this in machine learning by setting a low threshold for predicting positives which increases the number of true positives and false positives, but allows us to balance the the costs of being correct and incorrect.

**We can vary the classification threshold for our model to get different predictions. What happens? How is accuracy impacted? Does it matter?**

In [ ]:
cm = confusion_matrix(y_true=y_test, y_pred=logit_pred_proba > .3)

# use .ravel() properly
tn, fp, fn, tp = cm.ravel()

# print the result
print(cm)

**Compute accuracy**

In [ ]:
# Answer - compute accuracy
acc = (tn+tp)/(tp+tn+fp+fn)
print(f'Accuracy (TN+TP)/Total = {acc*100}%')

In [ ]:
tn/(tn+fp)

### The Accuracy Paradox

Accuracy is a very intuitive metric — it's a lot like an exam score where you get total correct/total attempted. However, accuracy is often a poor metric in application. There are many reasons for this:
- Imbalanced problems problems with 95% positives in the baseline will have 95% accuracy even with no predictive power.
  - This is the paradox; pursuing accuracy often means predicting the most common class rather than doing the most useful work.
- Applications often have uneven penalties and rewards for true positives and false positives.
- Ranking predictions in the correct order be more important than getting them correct.
- In many case we need to know the exact probability of a positives and negatives.
  - To calculate an expected return.
  - To triage observations that are borderline positive.

**Some of the most useful metrics for addressing these problems are:**
    
- **Classification accuracy/error**
  - Classification accuracy is the percentage of correct predictions (higher is better).
  - Classification error is the percentage of incorrect predictions (lower is better).
  - Easiest classification metric to understand.
- **Confusion matrix**
  - Gives you a better understanding of how your classifier is performing.
  - Allows you to calculate sensitivity, specificity, and many other metrics that might match your business objective better than accuracy.
  - Precision and recall are good for balancing misclassification costs.
- **ROC curves (tpr vs fpr) and area under a curve (AUC)**
  - Good for ranking and prioritization problems.
  - Allows you to visualize the performance of your classifier across all possible classification thresholds, thus helping you to choose a threshold that appropriately balances sensitivity and specificity.
  - Still useful when there is high class imbalance (unlike classification accuracy/error).
  - Harder to use when there are more than two response classes.
- **Log loss**
  - Most useful when well-calibrated predicted probabilities are important to your business objective.
    - Expected value calculations
    - Triage

The good news is that these are readily available in Python and R, and are usually easy to calculate once you know about them.

<a id="samples"></a>
## OPTIONAL: How Many Samples Are Needed?

We often ask how large our data set should be to achieve a reasonable logistic regression result. Below, a few methods will be introduced for determining how accurate the resulting model will be.

### Rule of Thumb

**Quick:** At least 100 samples total. At least 10 samples per feature.

**Formula method:**
1. Find the proportion $p$ of positive cases and negative cases. Take the smaller of the two. 
    - Ideally, you want 50/50 for a proportion of 0.5.
    - Example: Suppose we are predicting "male" or "female". Our data is 80% male, 20% female. 
        - So, we choose the proportion $p = 0.2$ since it is smaller.

2. Find the number of independent variables $k$.
    - Example: We are predicting gender based on the last letter of the first name, giving us 26 indicator columns for features. So, $k = 26$.

3. Let the minimum number of cases be $N = \frac{10k}{p}$. The minimum should always be set to at least $100$.
    - Example: Here, $N = 10*26 / 0.2 = 1300$. So, we would need 1300 names (supposing 80% are male).


Both methods from: Long, J. S. (1997). *Regression Models for Categorical and Limited Dependent Variables*. Thousand Oaks, CA: Sage Publications.


### Statistical Testing

Logistic regression is one of the few machine learning models where we can obtain comprehensive statistics. By performing hypothesis testing, we can understand whether we have sufficient data to make strong conclusions about individual coefficients and the model as a whole. A very popular Python library which gives you these statistics with just a few lines of code is [statsmodels](http://www.statsmodels.org/dev/index.html).


### Power Analysis

As you may suspect, many factors affect how statistically significant the results of a logistic regression are. The art of estimating the sample size to detect an effect of a given size with a given degree of confidence is called power analysis.

Some factors that influence the accuracy of our resulting model are:

+ Desired statistical significance (p-value)
+ Magnitude of the effect
    - It is more difficult to distinguish a small effect from noise. So, more data would be required!
+ Measurement precision
+ Sampling error
    - An effect is more difficult to detect in a smaller sample.
+ Experimental design

So, many factors, in addition to the number of samples, contribute to the resulting statistical power. Hence, it is difficult to give an absolute number without a more comprehensive analysis. This analysis is out of the scope of this lesson, but it is important to understand some of the factors that affect confidence.

## Lesson Review
- **Logistic regression**
  - What kind of machine learning problems does logistic regression address?
  - What do the coefficients in a logistic regression represent? How does the interpretation differ from ordinary least squares? How is it similar?
  
- **The confusion matrix**
  - How do true positive rate and false positive rate help explain accuracy?
  - Why might one classification metric be more important to tune than another? Give an example of a business problem or project where this would be the case.